# Decision tree

A tree generated from training data, that follows a set of if-then rules to reach a decision

Decision tree allows to make decision by following a top-down graph

## ID3 algorithms

1. Have a dataset with attributes (features) and class (dependent variables)
2. Pick the best attribute and the split values for this best attribute BASED ON ONFORMATION GAIN
3. Split the dataset based on this best attribute's split value (e.g. for discrete attributes, split into subgroup like red/blue/green ..., for continuous attributes, split into 'less than' and '>=') 
4. Make a decision tree node for each group
5. Back to step 3 until a stop rule is reached

https://medium.com/@hiromi_suenaga/machine-learning-1-lesson-1-84a1dc2b5236

**Why is decision tree is good**: Very few statistical assumptions are made before building a decision tree (colinearity, linear relationship, normally distributed, linear interactions ...). You don't even need to one-hot encoding categorical features.

# Random Forest

- Use a combination of decision trees to make prediction
- Trivially parallelizable: linearly scale with # of CPUs you have

## Build a single tree

1. Convert categorical features to numerical features using label encoding
2. Initiate score of tree's root node to float('inf')
3. Loop through a subset of features (based on max_features parameters)
    - For each feature, sort the column and loop through each value: find the value that splits dataset into 2 that results in lowest score (be mindful of min_leaf: smallest size of subtree allowed. Sometimes no split is made due to min_leaf). If lowest score is reached, save that feature ID and that feature's value
4. Save 2 list of data indices: one for the left subtree and one for the right subtree, then create these 2 subtrees by recursively go back to 2 for each subtree until min_leaf is reached

## Build a list of these trees (Random forest)

1. Create a list of n single tree (above), each tree contains only a subset of data using random permutation (with replacement)
2. To predict, average out n predictions from n trees

# Why random forest works well and avoid overfitting

- For 1 tree, it is easy to overfit (since it searches for best split value using greedy methods, and it's hard to choose the right max_depth or max_leaf params to avoid overfitting
- For a lot of trees, each tree only allows to look at a subset of data (or different random samples). Yes each tree will still be susceptible to overfitting, and **they will overfit in DIFFERENT ways (errors are random/errors are NOT CORRELATED TO EACH OTHER)**. But when we average their results, their errors should cancel out, resulting in a generalized model

To achieve this, make sure each trees are truly different from each other, meaning
- Correlation of each tree (either the split, or the error it generages) is as low as possible
- Every single tree should have high predictive power (THOUGH, The research community found that **the more important thing seems to be creating uncorrelated trees rather than more accurate trees**)

=> **Reduce # of sample for each trees (so that tree won't be able to remember data pattern) or max_features (so tree won't use some dominant features to make prediction)** will reduce predictive power for each tree, but **when bagging (in Random Forest), tree error correlation will be significantly lower.**

=> Also **Extra Tree can help with producing uncorrelated tree**: ExtraTreeClassifier won't try every split for each feature to find best split; it **randomly tried a few splits**, so tree will be crappier but surely uncorrelated, training is faster and **more uncorrelated tree can be built**

In [6]:
# how sklearn random forest gets sample of dataset for each tree with bootstrap option (select with/without replacement)
import numpy as np
# for dataset size 10
temp = np.random.choice(10,10)
print(temp) #bootstrap=True
print([i for i in range(10) if i not in temp]) #out of bag data

print('-'*10)
print(np.random.choice(10,10,replace=False)) #bootstrap=False

[1 2 2 0 2 5 1 7 0 4]
[3, 6, 8, 9]
----------
[2 1 3 6 8 5 4 9 0 7]


## Random forest and time-series data

Random forest doesn't work well with time-related features

- **All RF can do well is AVERAGE DATA THAT IT HAS ALREADY SEEN**; RF cannot extrapolate or calculate data that it hasn't seen before ('unseen' data)
    - In a sense, Random forest is similar to nearest neighbors such that **RF just return the average of a bunch of other points that it already observe**
    
- RF will treat 'future' time features as 'unseen' feature, and this is bad especially when we use later time for validation set. E.g: training data before 2010, validation data after 2010. Random forest never learn about any value after 2010, so any value that our of the year range, such as year 2011,2012 … all means the same to random forest: >2010, thus **returns the same split for all of these 'new' years (especially bad if this year features is important and end up near the root).**

=> Solution: use extrapolation technique **(to remove unnecessary time-related features, thus force RF to use some other non-time data/feature to make prediction => model will be less sensitive to time and generalize well with future data)**
-	**Time-series RF’s oob_score (for bootstrap=True) is not desirable** because out of bag data (that is not included in training) is not guaranteed to be later than data in training set (due to random sampling) => oob_score is normally higher, but it’s overfitting since data is not at later time

-	**What might be a problem with using RF for time-series data but it's not**: since for each tree, a subset of data is taken (from sampling with or without replacement, aka bootstrapping), you might think tree might not take into account toe consecutive nature of time, i.e. **tree won’t take into account n-consecutive days**, since data is randomly permuted, so it can only take non consecutive days t-7, t-3, t-1 …. But this is only for 1 tree. A random forest takes different subset of data for training, and it will have an insight on these consecutive days to when combining these predictions



More info and tree intepretation: https://github.com/anhquan0412/pandas_practice/blob/master/fraud_ml.ipynb

# Cool way to visualize decision trees and the idea of bagging in RF

Source: http://structuringtheunstructured.blogspot.com/2017/11/coloring-with-random-forests.html

We have a dataset with X and Y are independent variables (on XY axis) and dependent var is continuous value that depicts color

![](images/Selection_018.png)

Using a single decision tree, we can visualize best split for each tree level. For example at level 1, best split is at X=0.9, and we visualize it as a vertical line going through 0.9
![](images/Selection_019.png)

2nd level splits: (left tree where X<0.9) split at X=0.65, (right tree where X>=0.9) split at Y=0.87
![](images/Selection_020.png)

Eventually we can visualize all the splits of the tree
![](images/Selection_021.png)

For prediction using this tree, we just average the continuous color value for each box
![](images/Selection_022.png)

We repeated this for 3 other trees, resampling data for each tree's training
![](images/Selection_023.png)

And final prediction using all 4 trees = average these 4 trees' predictions
![](images/Selection_024.png)

This is a technique originally from a machine learning researchers